In [1]:
from sdv.datasets.demo import download_demo

data, metadata = download_demo('single_table', 'adult')


In [2]:
import sys
import os

sys.path.insert(0, '../') 

from synd import TGAN, CTGAN
from synd.datasets import SingleTable

discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'label',
]

batch_size = 200
epochs = 3
device = 'cuda:0'

dataset = SingleTable(data, metadata.to_dict(), discrete_columns=discrete_columns)
dataset.fit()

tgan = TGAN(batch_size=batch_size, device=device)
ctgan = CTGAN(batch_size=batch_size, device=device)


[2023-05-12 23:40:57,984] [synd.datasets.single_table] [DEBUG	] setting dataset name to: SingleTable20230512-21:40:57
[2023-05-12 23:40:57,985] [synd.datasets.single_table] [DEBUG	] creating `DataTransformer`
[2023-05-12 23:40:57,985] [synd.datasets.single_table] [DEBUG	] fitting `DataTransformer`
[2023-05-12 23:41:21,194] [synd.datasets.single_table] [DEBUG	] creating `DataSampler`


In [3]:
ctgan.fit(dataset, epochs=epochs)


Epoch		G loss		C loss
01		1.6104		3.4633
02		1.1318		0.1998
03		0.5073		0.0721


In [4]:
tgan.fit(dataset, epochs=epochs)


Epoch		G loss		C loss
01		-0.0152		11.2482
02		-0.0133		0.1753
03		-0.0138		0.1167


In [5]:
ctgan_sampleset = ctgan.sample(1200, dataset, sampleset_name='CTGAN-samples')


In [6]:
tgan_sampleset = tgan.sample(1200, dataset, sampleset_name='TGAN-samples')


In [7]:
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()
report.generate(data, ctgan_sampleset._data, metadata.to_dict())
report.generate(data, tgan_sampleset._data, metadata.to_dict())



Creating report: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.88it/s]



Overall Quality Score: 54.88%

Properties:
Column Shapes: 60.77%
Column Pair Trends: 48.99%


Creating report: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.07it/s]



Overall Quality Score: 76.66%

Properties:
Column Shapes: 79.73%
Column Pair Trends: 73.6%
